In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

In [ ]:
# definir as labels finais
labels = []

Lying down 
Sitting 
Walking 
Running 
Bicycling
Sleeping
Lab work
In class
In a meeting 
At main workplace
Indoors 
Outside
In a car 
On a bus 
Drive (I'm the driver) 
Drive (I'm a passenger) 
At home 
At a restaurant
Phone in pocket 
Exercise 
Cooking
Shopping
Strolling 
Drinking (alcohol) 
Bathing - shower 
Cleaning
Laundry 
Washing dishes 
Watching TV
Surfing the internet 
At a party 
At a bar 
At the beach 
Singing
Talking 
Computer work 
Eating
Toilet 
Grooming 
Dressing 
At the gym
Stairs - going up
Stairs - going down
Elevator 
Standing 
At school
Phone in hand
Phone in bag
Phone on table 
With co-workers 
With friends

In [ ]:
# montar os 20 folds da seguinte maneira:
# percorra toda a pasta onde tem os datasets dos usuarios e pegue os `ids`
# selecione 40 aleatoriamente, 20x
# carregue apenas esses 40 datasets (delete-os da memoria ao final) e treine um modelo unificado

In [ ]:
# faca uma otimizacao de parametros por algum tipo de grid ou random search (ou evolutivo, se der tempo)

In [ ]:
config = {'df_path': '../input/user1.features_labels.csv'}
har = utils.HAR(config)
har.dummy_test()